# Usando aprendizaje no supervisado para detectar nombre y apellido en datos de nombre completo sin procesar

Es un escenario común encontrarse en el desarrollo de sistemas con conjuntos de datos que no han separado las columnas de nombre completo, por ejemplo, en `first_name` y `last_name`.

Tal es el caso del dataset con el que vamos a trabajar, que corresponde a las declaraciones juradas patrimoniales de funcionarios públicos de Argentina.
Es un escenario común de preprocesamiento de texto, por lo que vamos a usar técnicas comunes para tales escenarios: TfIdf y algoritmos de clustering para encontrar los grupos automáticamente.

## Instalamos pandas y scikit-learn

In [1]:
!pip install pandas scikit-learn hdbscan matplotlib tabulate

## Importamos los módulos de sklearn que vamos a usar:

In [2]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.base import clone


## Importamos pandas y cargamos el csv con declaraciones juradas

In [3]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/nmercado1986/jupyter-notebooks/master/ddjj.csv')

## Un pequeño vistazo:

In [4]:
print(df['funcionario_apellido_nombre'].head())

0        ABA MARCELO ALFREDO
1    ABACA ALBERTO ALEJANDRO
2    ABACA ALBERTO ALEJANDRO
3     ABACA HECTOR ALEJANDRO
4       ABAD ALBERTO REMIGIO
Name: funcionario_apellido_nombre, dtype: object


## De esto solo nos interesan los nombres:

Los queremos estandarizar en minúsculas. Además, vamos a sacar los stop-words del tipo:
- de: *De Achával, Fernández de Kirchner, el prefijo De no es necesario para determinar si es un apellido o nombre* 
- d'
- del
- de la
- y

In [5]:
df = df[df.tipo_declaracion_jurada_id==1]

import re
df['funcionario_apellido_nombre'] = df['funcionario_apellido_nombre'].apply(lambda s: s.lower())
df['funcionario_apellido_nombre'] = df['funcionario_apellido_nombre'].apply(lambda s: re.sub(r'\b(d|d\'|\'|de|del|y|la|las|los)\b\s*', '', s))


## Separamos los nombres en 3 features distintos de acuerdo al 1er, 2do y 3er token

Para no tener que estar lidiando con problemas de tipo por `None` en esos casos donde no hay tercer token (gente sin nombre del medio), lo reemplazamos con un token `--SIN NOMBRE--`

In [6]:
dftrain = pd.DataFrame()
dftrain['funcionario_apellido_nombre'] = df['funcionario_apellido_nombre']
dftrain['palabra0'] = df['funcionario_apellido_nombre'].apply(lambda s: s.split(' ')[0])
dftrain['palabra1'] = df['funcionario_apellido_nombre'].apply(lambda s: s.split(' ')[1])
dftrain['palabra2'] = df['funcionario_apellido_nombre'].apply(lambda s: s.split(' ')[2] if len(s.split(' ')) > 2 else "--SIN NOMBRE--")
dftrain['count'] = df['funcionario_apellido_nombre'].apply(lambda s: len(s.split(' ')))

###  Luego armamos un vocabulario con todos los nombres propios juntos. 
Este vocabulario se lo pasamos como parámetro a tf-idf.

In [7]:
todosLosNombres = pd.concat([dftrain['palabra0'], dftrain['palabra1'], dftrain['palabra2']], ignore_index=True).dropna().unique()
todosLosNombres = list(map(lambda s: s.lower(), todosLosNombres))

## Aplicando tf-idf y centrando features
Usamos tf-idf para calcular la frecuencia de cada nombre propio en cada una de las posiciones y en general en todo el listado de nombres

In [8]:
pipe = Pipeline([
    ('count', CountVectorizer(vocabulary=list(todosLosNombres))),
    ('idf', TfidfTransformer(norm=None, 
                             sublinear_tf=True, 
                             smooth_idf=True)),
    #('scaler', MaxAbsScaler())
])
pipe0 = clone(pipe)
pipe1 = clone(pipe)
pipe2 = clone(pipe)
pipe.fit(dftrain['funcionario_apellido_nombre'])
pipe0.fit(dftrain['palabra0'])
pipe1.fit(dftrain['palabra1'])
pipe2.fit(dftrain['palabra2'])

dftrain['tf_idf_palabra1'] = RobustScaler().fit_transform(
    np.amax(pipe.transform(dftrain['palabra1']).toarray(), axis=1).reshape(-1, 1))

dftrain['tf_idf_palabra1_in0'] = RobustScaler().fit_transform(
    np.amax(pipe0.transform(dftrain['palabra1']).toarray(), axis=1).reshape(-1, 1)
    / np.amax(pipe.transform(dftrain['palabra1']).toarray(), axis=1).reshape(-1, 1))
dftrain['tf_idf_palabra1_in1'] = RobustScaler().fit_transform(
    np.amax(pipe1.transform(dftrain['palabra1']).toarray(), axis=1).reshape(-1, 1)/
    np.amax(pipe.transform(dftrain['palabra1']).toarray(), axis=1).reshape(-1, 1))
dftrain['tf_idf_palabra1_in2'] = RobustScaler().fit_transform(
    np.amax(pipe2.transform(dftrain['palabra1']).toarray(), axis=1).reshape(-1, 1) / 
    np.amax(pipe.transform(dftrain['palabra1']).toarray(), axis=1).reshape(-1, 1))

dftrain['tf_idf_solo_dos'] = StandardScaler().fit_transform((dftrain['palabra2'] == "--SIN NOMBRE--").to_numpy().reshape(-1, 1))

# Algunos features que no vamos a extraer
#dftrain['tf_idf_palabra1'] = RobustScaler().fit_transform(np.amax(pipe.transform(dftrain['palabra1']).toarray(), axis=1).reshape(-1, 1))
#dftrain['tf_idf_palabra0_in1'] = StandardScaler().fit_transform(np.amax(pipe1.transform(dftrain['palabra0']).toarray(), axis=1).reshape(-1, 1))
#dftrain['tf_idf_palabra0_in2'] = (np.amax(pipe2.transform(dftrain['palabra0']).toarray(), axis=1))

#dftrain['tf_idf_palabra1_comb_1_1'] = StandardScaler().fit_transform((dftrain['tf_idf_palabra1_in0'] / dftrain['tf_idf_palabra1_in2']).to_numpy().reshape(-1, 1))

#dftrain['tf_idf_palabra1_in1'] = RobustScaler().fit_transform(np.amax(pipe1.transform(dftrain['palabra1']).toarray(), axis=1).reshape(-1, 1))

#dftrain['tf_idf_palabra2_in0'] = StandardScaler().fit_transform(np.amax(pipe0.transform(dftrain['palabra2']).toarray(), axis=1).reshape(-1, 1))
#dftrain['tf_idf_palabra2_in1'] = StandardScaler().fit_transform(np.amax(pipe1.transform(dftrain['palabra2']).toarray(), axis=1).reshape(-1, 1))
#dftrain['tf_idf_palabra2_in2'] = StandardScaler().fit_transform(np.amax(pipe2.transform(dftrain['palabra2']).toarray(), axis=1).reshape(-1, 1))
dftrain['tf_count'] = StandardScaler().fit_transform(dftrain['count'].to_numpy().reshape(-1, 1))

#print((100 * dftrain['tf_idf_palabra1_in0'] / dftrain['tf_idf_palabra1_in2']).to_numpy().reshape(-1, 1))

dftrain['tf_idf_palabra1_cross_0_1'] = RobustScaler().fit_transform(
    np.amax(pipe0.transform(dftrain['palabra1']).toarray(), axis=1).reshape(-1, 1) /
    np.amax(pipe1.transform(dftrain['palabra1']).toarray(), axis=1).reshape(-1, 1))



dftrain['tf_idf_palabra1_cross_1_2'] = RobustScaler().fit_transform(
    np.amax(pipe1.transform(dftrain['palabra1']).toarray(), axis=1).reshape(-1, 1) /
    np.amax(pipe2.transform(dftrain['palabra1']).toarray(), axis=1).reshape(-1, 1))

dftrain['tf_idf_palabra1_cross_0_2'] = RobustScaler().fit_transform(
    np.amax(pipe0.transform(dftrain['palabra1']).toarray(), axis=1).reshape(-1, 1) /
    np.amax(pipe2.transform(dftrain['palabra1']).toarray(), axis=1).reshape(-1, 1))

# Remove NAs
dftrain = dftrain[~dftrain.tf_idf_palabra1_in0.isna()]
dftrain = dftrain[~dftrain.tf_idf_palabra1_in1.isna()]
dftrain = dftrain[~dftrain.tf_idf_palabra1_in2.isna()]
dftrain = dftrain[~dftrain.tf_idf_palabra1_cross_0_1.isna()]
dftrain = dftrain[~dftrain.tf_idf_palabra1_cross_0_2.isna()]
dftrain = dftrain[~dftrain.tf_idf_palabra1_cross_1_2.isna()]

dftrain.columns


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in true_divide


Index(['funcionario_apellido_nombre', 'palabra0', 'palabra1', 'palabra2',
       'count', 'tf_idf_palabra1', 'tf_idf_palabra1_in0',
       'tf_idf_palabra1_in1', 'tf_idf_palabra1_in2', 'tf_idf_solo_dos',
       'tf_count', 'tf_idf_palabra1_cross_0_1', 'tf_idf_palabra1_cross_1_2',
       'tf_idf_palabra1_cross_0_2'],
      dtype='object')

## Por ejemplo, buscamos las filas con el nombre 'gomez'

En cada fila, obtenermos la frecuencia inversa del nombre 'gomez' dependiendo de dónde aparece el término. 
El indicador `tf_idf_palabra1_in0` nos muestra qué tan inusual es 'gomez' se encuentre en la primera posición.
El indicador `tf_idf_palabra1_in2` nos muestra qué tan inusual es que 'gomez' se encuentre en la tercera posición. 



In [9]:
dftrain[dftrain.funcionario_apellido_nombre.str.contains('gomez')].head(50)

,funcionario_apellido_nombre,palabra0,palabra1,palabra2,count,tf_idf_palabra1,tf_idf_palabra1_in0,tf_idf_palabra1_in1,tf_idf_palabra1_in2,tf_idf_solo_dos,tf_count,tf_idf_palabra1_cross_0_1,tf_idf_palabra1_cross_1_2,tf_idf_palabra1_cross_0_2
13848,corzo gomez clara isabel,corzo,gomez,clara,4,0.117665,-1.477732,4.048255,6.635483,-0.307138,2.140476,-1.981618,-1.260492,-2.190563
14140,criante gomez yesica ivonne,criante,gomez,yesica,4,0.117665,-1.477732,4.048255,6.635483,-0.307138,2.140476,-1.981618,-1.260492,-2.190563
16593,diaz gomez arturo leon,diaz,gomez,arturo,4,0.117665,-1.477732,4.048255,6.635483,-0.307138,2.140476,-1.981618,-1.260492,-2.190563
18765,farias gomez matias federico,farias,gomez,matias,4,0.117665,-1.477732,4.048255,6.635483,-0.307138,2.140476,-1.981618,-1.260492,-2.190563
19255,fernandez gomez rodrigo daniel,fernandez,gomez,rodrigo,4,0.117665,-1.477732,4.048255,6.635483,-0.307138,2.140476,-1.981618,-1.260492,-2.190563
21621,gallardo gomez felix gaston,gallardo,gomez,felix,4,0.117665,-1.477732,4.048255,6.635483,-0.307138,2.140476,-1.981618,-1.260492,-2.190563
22204,garcia gomez gonzalo antonio,garcia,gomez,gonzalo,4,0.117665,-1.477732,4.048255,6.635483,-0.307138,2.140476,-1.981618,-1.260492,-2.190563
23251,gil gomez maria eugenia,gil,gomez,maria,4,0.117665,-1.477732,4.048255,6.635483,-0.307138,2.140476,-1.981618,-1.260492,-2.190563
23898,gomez adrian nestor,gomez,adrian,nestor,3,-0.035149,0.291944,0.825245,-0.586360,-0.307138,-0.050966,0.028244,0.976041,0.580094
23899,gomez aguirre mauricio maximiliano,gomez,aguirre,mauricio,4,0.725953,-1.482720,3.139098,3.778691,-0.307138,2.140476,-1.911972,-0.540887,-1.981675


# Agrupando usando KMeans

KMeans va a particionar nuestros datos, asignando obligadamente un cluster a cada elemento.




In [19]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import silhouette_score
import math
#dbs = DBSCAN(eps=1, min_samples=5)
#from sklearn.cluster import AgglomerativeClustering

pgrid = ParameterGrid({ 
    'n_clusters' : [ 10, 12, 15, 20, 25, 30, 35 ]
    })

pca = PCA(n_components=5)
#features = pca.fit_transform(dftrain.filter(regex='tf_.*').sample(10000))

features = pca.fit_transform(dftrain.filter(regex='tf_.*'))

best = None
bestScore = -np.Inf
for params in pgrid:

  dbs = KMeans(**params, random_state=0)
  dbs.fit(features)


  try:
    score = math.floor(100.0 * silhouette_score(features, dbs.labels_)) / 100
    print(params, score)
    if score >= bestScore:
      best = dbs
      bestScore = score
  except Exception as e:
    print(e)
    continue
  
dftrain['cluster'] = best.fit_predict(dftrain.filter(regex='tf_.*'))

print(dftrain[dftrain.cluster == -1])


{'n_clusters': 10} 0.36
{'n_clusters': 12} 0.39
{'n_clusters': 15} 0.42
{'n_clusters': 20} 0.42
{'n_clusters': 25} 0.44
{'n_clusters': 30} 0.44
{'n_clusters': 35} 0.45
Empty DataFrame
Columns: [funcionario_apellido_nombre, palabra0, palabra1, palabra2, count, tf_idf_palabra1, tf_idf_palabra1_in0, tf_idf_palabra1_in1, tf_idf_palabra1_in2, tf_idf_solo_dos, tf_count, tf_idf_palabra1_cross_0_1, tf_idf_palabra1_cross_1_2, tf_idf_palabra1_cross_0_2, cluster]
Index: []


In [16]:
dftrain['cluster'] = best.fit_predict(dftrain.filter(regex='tf_.*'))


## Previsualizamos nuestros grupos

HDBSCAN nos va a devolver un ID de cluster por fila. Veamos cuántos elementos tiene cada clúster, y la cantidad total de clústeres encontrados.
El clúster -1 corresponde a los outliers.

In [20]:
print(dftrain['cluster'].value_counts())
print(dftrain['cluster'].max())

7     4267
8     3963
4     2529
13    2383
30    2275
16    2157
32    2114
0     2108
2     1927
28    1749
9     1509
14    1401
23    1325
34    1255
21    1207
31    1184
19    1181
1     1142
6     1067
20     975
10     759
29     637
3      614
24     584
18     528
15     506
26     485
22     441
27     353
11     325
25     313
17     274
33     260
5      245
12     161
Name: cluster, dtype: int64
34


**Luego importamos `matplotlib` para visualizar los grupos que se formaron sobre el componente extraído.**

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

plt.rcParams['figure.figsize'] = [20, 10]


fig = plt.figure()

def plotApellidos(dftrain, apellido, ax):
    """ Plotea los nombres de un apellido vs la totalidad del dataset en 2d
    """
    dfsample = dftrain.sample(1000)
    colors = ['red', 'green', 'blue', 'yellow', 'pink', 'orange']
        
    """ TSNE tarda mucho con muestras grandes """
    tsne = TSNE(n_iter=1000)
    proj = tsne.fit_transform(dfsample.filter(regex = 'tf_.*'))
    dfsample['tsne0'] = proj[:,0]
    dfsample['tsne1'] = proj[:,1]

    dfApellido = dfsample[dfsample.funcionario_apellido_nombre.str.contains(apellido)]


    
    #projApellido = tsne.fit_transform(dfApellido.filter(regex = 'tf_.*'))
    #dfApellido['pca0'] = projApellido[:,0]
    #dfApellido['pca1'] = projApellido[:,1]
    
    ax.scatter(dfsample['tsne0'], dfsample['tsne1'], alpha=.5, s=5, c= dfsample['cluster'].apply(lambda i: colors[ i % len(colors) ]))
    ax.scatter(dfApellido['tsne0'], dfApellido['tsne0'], s=20, c= dfApellido['cluster'].apply(lambda i: colors[ i % len(colors) ]))

    for id, row in dfApellido.iterrows():
        ax.annotate(row['funcionario_apellido_nombre'], (row['tsne0'], row['tsne1']))


    plt.show()
    
    
 

plotApellidos(dftrain, 'gomez', fig.add_subplot('111'))

Veamos cómo fue catalogado cada uno de los Gómez.
A partir del gráfico, podemos ver que aquellos `Gómez` donde este apellido es uno de varios están en el grupo más a la izquierda del eje X
Incluso, vemos el caso de `Gómez Albano Emmanuel`, lo que tiene sentido porque el nombre propio `Albano` es difícil de identificar como nombre o como apellido.
El usar un sólo componente nos permitió establecer una diferencia muy tajante.
Un apellido o muchos, sin especificar demasiado pequeñas agrupaciones como si los nombres tienen 2 o 3 palabras, si son de apellidos comunes o poco comunes, etc. Sólo 65 elementos no fueron reconocidos.

Como vemos, los Gómez quedaron separados en los dos grupos que queríamos: **aquelles con un solo apellido** y **aquelles con dos**.
Las filas de doble apellido pertenecen al cluster 0, tanto en los casos donde Gómez era el primer apellido (Gómez Alcaraz, Gómez Aguirre) como cuando era el segundo (Gil Gómez, Díaz Gómez).


In [21]:
dfgomez = dftrain[dftrain.funcionario_apellido_nombre.str.contains('gomez')]
dfgomez[['funcionario_apellido_nombre', 'cluster']].head(50)

,funcionario_apellido_nombre,cluster
13848,corzo gomez clara isabel,17
14140,criante gomez yesica ivonne,17
16593,diaz gomez arturo leon,17
18765,farias gomez matias federico,17
19255,fernandez gomez rodrigo daniel,17
21621,gallardo gomez felix gaston,17
22204,garcia gomez gonzalo antonio,17
23251,gil gomez maria eugenia,17
23898,gomez adrian nestor,2
23899,gomez aguirre mauricio maximiliano,5


### Otros apellidos

Todo muy bien con los Gómez, quedaron bien catalogados, pero veamos otros apellidos, comunes y no tanto.
Veamos por ejemplo el apellido `Acevedo`, un poco menos común.

In [22]:

dfacevedo = dftrain[dftrain.funcionario_apellido_nombre.str.contains('acevedo')]
dfacevedo[['funcionario_apellido_nombre', 'cluster']]

,funcionario_apellido_nombre,cluster
219,acevedo alberto enrique,11
223,acevedo arnaldo daniel,9
224,acevedo barbara yanel,21
225,acevedo carlos adrian,7
226,acevedo cesar alberto,2
227,acevedo daniel alberto jesus,27
228,acevedo delia beatriz,21
230,acevedo diaz eduardo alejandro,17
231,acevedo eduardo horacio,13
232,acevedo gabriel esteban,2


In [23]:
grupos = dftrain.groupby('cluster').apply(lambda x: x.sample(3))
with pd.option_context('display.max_rows', None):
  print(grupos)

                       funcionario_apellido_nombre  ... cluster
cluster                                             ...        
0       30204                  llarin victor dario  ...       0
        26371             hermida mariano leonardo  ...       0
        10117            carambia victor guillermo  ...       0
1       34055                     mazzoni agustina  ...       1
        17483                    dunayevich julian  ...       1
        41552                    petrecca federico  ...       1
2       47931              salomon horacio eduardo  ...       2
        10657                 carrizo cesar miguel  ...       2
        28890          landriscini susana graciela  ...       2
3       3570             arturo mateos maria valle  ...       3
        30267           lo presti veronica amneris  ...       3
        16428              di stefano lorena paola  ...       3
4       50416           snopek maria eugenia tulia  ...       4
        53187              troncoso mari

In [ ]:
plotApellidos(dftrain, 'acevedo', plt.figure().add_subplot('111'))

In [ ]:
dfpereyra = dftrain[dftrain.funcionario_apellido_nombre.str.contains('pereyra')].head(50)
dfpereyra[['funcionario_apellido_nombre', 'cluster']]

In [ ]:
plotApellidos(dftrain, 'pereyra', plt.figure().add_subplot('111'))

**Finalmente, exportamos los dos grupos por separado, uno para la gente con doble apellido y otro para la gente con uno solo:**

In [25]:
dftrain[dftrain.cluster.isin([3,  5,  6, 17, 25, 26, 33, 34])].to_csv('dos_o_mas_apellidos.csv')
dftrain[~dftrain.cluster.isin([3,  5,  6, 17, 25, 26, 33, 34])].to_csv('un_apellido.csv')

Los archivos están disponibles para descargar en el notebook con el nombre de [un_apellido.csv](/edit/un_apellido.csv) y [doble_apellido.csv](/edit/doble_apellido.csv).